# Velocyto flowchart

Velocyto -> command(run, run10x, run_smartseq2, run_dropest, dropest_bc_correct) with option (implemented by click package)
-> input data(bam/sam/bed....) to run function -> input data to _run(actually this function works) function -> return result from _run -> return result

In [ ]:
#velocyto.py

import click # make py file available by line command
import logging # tracking
import sys
from typing import Any # making new data types 
from collections import OrderedDict # import user_defined dictionary -> preserve the order commands were added.
from .run import run
from .run10x import run10x
from .run_smartseq2 import run_smartseq2
from .run_dropest import run_dropest
from .dropest_bc_correct import dropest_bc_correct
import velocyto._version


class NaturalOrderGroup(click.Group):
    """Command group trying to list subcommands in the order they were added.
    Make sure you initialize the `self.commands` with OrderedDict instance.
    With decorator, use::
        @click.group(cls=NaturalOrderGroup, commands=OrderedDict())
    """

    def list_commands(self, ctx: Any) -> Any:
        """List command names as they are in commands dict.
        If the dict is OrderedDict, it will preserve the order commands
        were added.
        """
        return self.commands.keys() # command list


@click.version_option(version=velocyto._version.__version__)
@click.group(cls=NaturalOrderGroup, commands=OrderedDict(), context_settings=dict(max_content_width=300, terminal_width=300))
def cli() -> None:
    # NOTE: here is a good place to add a comand to control verbosity/logging-level
    logging.basicConfig(stream=sys.stdout, format='%(asctime)s - %(levelname)s - %(message)s', level=logging.DEBUG)
    return


@click.group(cls=NaturalOrderGroup, commands=OrderedDict(), context_settings=dict(max_content_width=300, terminal_width=300))
def tools() -> None:
    """helper tools for velocyto
    """
    return

tools.add_command(dropest_bc_correct)
cli.add_command(run)
cli.add_command(run10x)
cli.add_command(run_dropest)
cli.add_command(run_smartseq2)
cli.add_command(tools)

In [ ]:
# run.py -> from velocyto command

import sys
import os
import glob
# The glob module finds all the pathnames matching 
# a specified pattern according to the rules used by the Unix shell, 
# although results are returned in arbitrary order. 
import re # regular expression
import click
import numpy as np
import random
import string
import logging
from typing import *
import velocyto as vcy
from ._run import _run

# logging.basicConfig(stream=sys.stdout, format='%(asctime)s - %(levelname)s - %(message)s', level=logging.DEBUG)


def id_generator(size: int=6, chars: str=string.ascii_uppercase + string.digits) -> str:
    return ''.join(random.choice(chars) for _ in range(size))


@click.command(short_help="Runs the velocity analysis outputting a loom file")
@click.argument("bamfile", nargs=-1, required=True,
                type=click.Path(exists=True,
                                file_okay=True,
                                dir_okay=False,
                                readable=True,
                                resolve_path=True))
@click.argument("gtffile",
                type=click.Path(exists=True,
                                file_okay=True,
                                dir_okay=False,
                                readable=True,
                                resolve_path=True))

# options
@click.option("--bcfile", "-b",
              help="""Valid barcodes file, to filter the bam. If --bcfile is not specified all the cell barcodes will be included.
              Cell barcodes should be specified in the bcfile as the `CB` tag for each read""",
              default=None,
              show_default=True,
              type=click.Path(resolve_path=True,
                              file_okay=True,
                              dir_okay=False,
                              readable=True))
@click.option("--outputfolder", "-o",
              help="Output folder, if it does not exist it will be created.",
              default=None,
              type=click.Path(exists=False))
@click.option("--sampleid", "-e",
              help="The sample name that will be used to retrieve informations from metadatatable",
              default=None,
              type=click.Path(exists=False))
@click.option("--metadatatable", "-s",
              help="Table containing metadata of the various samples (csv formatted, rows are samples and cols are entries)",
              default=None,
              type=click.Path(resolve_path=True,
                              file_okay=True,
                              dir_okay=False,
                              readable=True))
@click.option("--mask", "-m",
              help=".gtf file containing intervals to mask",
              default=None,
              type=click.Path(resolve_path=True,
                              file_okay=True,
                              dir_okay=False,
                              readable=True))
@click.option("--onefilepercell", "-c",
              help="""If this flag is used every bamfile passed is interpreted as an independent cell, otherwise multiple files are interpreted as batch of different cells to be analyzed together.
              Important: cells reads should not be distributed over multiple bamfiles is not supported!! (default: off)""",
              default=False,
              is_flag=True)
@click.option("--logic", "-l",
              help="The logic to use for the filtering (default: Default)",
              default="Default")
@click.option("--without-umi", "-U",
              help="If this flag is used the data is assumed UMI-less and reads are counted instead of molecules (default: off)",
              default=False,
              is_flag=True)
@click.option("--umi-extension", "-u",
              help="""In case UMI is too short to guarantee uniqueness (without information from the ampping) set this parameter to `chr`, `Gene` ro `[N]bp`
              If set to `chr` the mapping position (binned to 10Gb intervals) will be appended to `UB` (ideal for InDrops+dropEst). If set to `Gene` then the `GX` tag will be appended to the `UB` tag.
              If set to `[N]bp` the first N bases of the sequence will be used to extend `UB` (ideal for STRT). (Default: `no`)""",
              default="no")
@click.option("--multimap", "-M",
              help="""Consider not unique mappings (not reccomended)""",
              default=False,
              is_flag=True)
@click.option("--samtools-threads", "-@",
              help="The number of threads to use to sort the bam by cellID file using samtools",
              default=16)
@click.option("--samtools-memory",
              help="The number of MB used for every thread by samtools to sort the bam file",
              default=2048)
@click.option("--dtype", "-t",
              help="The dtype of the loom file layers - if more than 6000 molecules/reads per gene per cell are expected set uint32 to avoid truncation (default run: uint32)",
              default="uint32")
@click.option("--dump", "-d",
              help="For debugging purposes only: it will dump a molecular mapping report to hdf5. --dump N, saves a cell every N cells. If p is prepended a more complete (but huge) pickle report is printed (default: 0)",
              default="0")
@click.option('--verbose', '-v',
              help="Set the vebosity level: -v (only warnings) -vv (warnings and info) -vvv (warnings, info and debug)",
              count=True, default=1)

# velocyto run -> input those files -> input at _run function
def run(bamfile: str, gtffile: str,
        bcfile: str, outputfolder: str,
        sampleid: str, metadatatable: str,
        mask: str, onefilepercell: bool, logic: str, without_umi: str, umi_extension: str, multimap: bool,
        samtools_threads: int, samtools_memory: int, dtype: str, dump: str, verbose: int,
        additional_ca: dict={}) -> None:
    """Runs the velocity analysis outputting a loom file
    BAMFILE bam file with sorted reads
    GTFFILE genome annotation file
    """
    return _run(bamfile=bamfile, gtffile=gtffile, bcfile=bcfile, outputfolder=outputfolder,
                sampleid=sampleid, metadatatable=metadatatable, repmask=mask, onefilepercell=onefilepercell,
                logic=logic, without_umi=without_umi, umi_extension=umi_extension, multimap=multimap, test=False, samtools_threads=samtools_threads,
                samtools_memory=samtools_memory, dump=dump, loom_numeric_dtype=dtype, verbose=verbose, additional_ca=additional_ca)

In [ ]:
#run10x.py

import sys
import os
import glob
import re
import gzip
import click
import numpy as np
import csv
from collections import defaultdict
import logging
from typing import *
import velocyto as vcy
from ._run import _run


logging.basicConfig(stream=sys.stdout, format='%(asctime)s - %(levelname)s - %(message)s', level=logging.DEBUG)


@click.command(short_help="Runs the velocity analysis for a Chromium Sample")
@click.argument("samplefolder",
                type=click.Path(exists=True,
                                file_okay=False,
                                dir_okay=True,
                                readable=True,
                                writable=True,
                                resolve_path=True))
@click.argument("gtffile",
                type=click.Path(exists=True,
                                file_okay=True,
                                dir_okay=False,
                                readable=True,
                                resolve_path=True))
@click.option("--metadatatable", "-s",
              help="Table containing metadata of the various samples (csv fortmated rows are samples and cols are entries)",
              default=None,
              type=click.Path(resolve_path=True,
                              file_okay=True,
                              dir_okay=False,
                              readable=True))
@click.option("--mask", "-m",
              help=".gtf file containing intervals to mask",
              default=None,
              type=click.Path(resolve_path=True,
                              file_okay=True,
                              dir_okay=False,
                              readable=True))
@click.option("--logic", "-l",
              help="The logic to use for the filtering (default: Default)",
              default="Default")
@click.option("--multimap", "-M",
              help="""Consider not unique mappings (not reccomended)""",
              default=False,
              is_flag=True)
@click.option("--samtools-threads", "-@",
              help="The number of threads to use to sort the bam by cellID file using samtools",
              default=16)
@click.option("--samtools-memory",
              help="The number of MB used for every thread by samtools to sort the bam file",
              default=2048)
@click.option("--dtype", "-t",
              help="The dtype of the loom file layers - if more than 6000 molecules/reads per gene per cell are expected set uint32 to avoid truncation (default run_10x: uint16)",
              default="uint16")
@click.option("--dump", "-d",
              help="For debugging purposes only: it will dump a molecular mapping report to hdf5. --dump N, saves a cell every N cells. If p is prepended a more complete (but huge) pickle report is printed (default: 0)",
              default="0")
@click.option('--verbose', '-v',
              help="Set the vebosity level: -v (only warinings) -vv (warinings and info) -vvv (warinings, info and debug)",
              count=True, default=1)
def run10x(samplefolder: str, gtffile: str,
           metadatatable: str, mask: str, logic: str, multimap: bool,
           samtools_threads: int, samtools_memory: int, dtype: str, dump: str, verbose: str) -> None:
    """Runs the velocity analysis for a Chromium 10X Sample
    10XSAMPLEFOLDER specifies the cellranger sample folder
    GTFFILE genome annotation file
    """

    # Check that the 10X analysis was run successfully
    if not os.path.isfile(os.path.join(samplefolder, "_log")):
        logging.error("This is an older version of cellranger, cannot check if the output are ready, make sure of this yourself")
    elif "Pipestance completed successfully!" not in open(os.path.join(samplefolder, "_log")).read():
        logging.error("The outputs are not ready")
    bamfile = os.path.join(samplefolder, "outs", "possorted_genome_bam.bam")

    bcmatches = glob.glob(os.path.join(samplefolder, os.path.normcase("outs/filtered_gene_bc_matrices/*/barcodes.tsv")))
    if len(bcmatches) == 0:
        bcmatches = glob.glob(os.path.join(samplefolder, os.path.normcase("outs/filtered_feature_bc_matrix/barcodes.tsv.gz")))
    if len(bcmatches) == 0:
        logging.error("Can not locate the barcodes.tsv file!")
    bcfile = bcmatches[0]
    
    outputfolder = os.path.join(samplefolder, "velocyto")
    sampleid = os.path.basename(samplefolder.rstrip("/").rstrip("\\"))
    assert not os.path.exists(os.path.join(outputfolder, f"{sampleid}.loom")), "The output already exist. Aborted!"
    additional_ca = {}
    try:
        tsne_file = os.path.join(samplefolder, "outs", "analysis", "tsne", "2_components", "projection.csv")
        if os.path.exists(tsne_file):
            tsne = np.loadtxt(tsne_file, usecols=(1, 2), delimiter=',', skiprows=1)
            additional_ca["_X"] = tsne[:, 0].astype('float32')
            additional_ca["_Y"] = tsne[:, 1].astype('float32')

        clusters_file = os.path.join(samplefolder, "outs", "analysis", "clustering", "graphclust", "clusters.csv")
        if os.path.exists(clusters_file):
            labels = np.loadtxt(clusters_file, usecols=(1, ), delimiter=',', skiprows=1)
            additional_ca["Clusters"] = labels.astype('int') - 1

    except Exception:
        logging.error("Some IO problem in loading cellranger tsne/pca/kmeans files occurred!")

    return _run(bamfile=(bamfile, ), gtffile=gtffile, bcfile=bcfile, outputfolder=outputfolder,
                sampleid=sampleid, metadatatable=metadatatable, repmask=mask, onefilepercell=False,
                logic=logic, without_umi=False, umi_extension="no", multimap=multimap, test=False, samtools_threads=samtools_threads,
                samtools_memory=samtools_memory, dump=dump, loom_numeric_dtype=dtype, verbose=verbose, additional_ca=additional_ca)